In [1]:
# 导入
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体为黑体
plt.rcParams['axes.unicode_minus'] = False  # 正确显示负号
import warnings
warnings.filterwarnings("ignore")
import xgboost as xgb
from xgboost import XGBClassifier

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import multilabel_confusion_matrix

In [3]:
# 读取数据
# df = pd.read_excel("222.xlsx")
df = pd.read_csv("test.csv", index_col=0) #M101 test
df.head()

,日期,时间,生产线编号,物料推送气缸推送状态,物料推送气缸收回状态,物料推送数,物料待抓取数,放置容器数,容器上传检测数,填装检测数,...,不合格数,物料推送装置故障1001,物料检测装置故障2001,填装装置检测故障4001,填装装置定位故障4002,填装装置填装故障4003,加盖装置定位故障5001,加盖装置加盖故障5002,拧盖装置定位故障6001,拧盖装置拧盖故障6002
0,1,9759,M101,1,1,1464,1464,491,490,488,...,0,1001,0,0,0,0,0,0,0,0
1,1,9760,M101,1,1,1464,1464,491,490,488,...,0,1001,0,0,0,0,0,0,0,0
2,1,9761,M101,1,1,1464,1464,491,490,488,...,0,1001,0,0,0,0,0,0,0,0
3,1,9762,M101,1,1,1464,1464,491,490,488,...,0,1001,0,0,0,0,0,0,0,0
4,1,9763,M101,1,1,1464,1464,491,490,488,...,0,1001,0,0,0,0,0,0,0,0


In [4]:
df = df.drop(['生产线编号'], axis=1)
df.set_index('时间', inplace=True)

# 建模
过采样或者使用原数据

In [5]:
df.columns

Index(['日期', '物料推送气缸推送状态', '物料推送气缸收回状态', '物料推送数', '物料待抓取数', '放置容器数', '容器上传检测数',
       '填装检测数', '填装定位器固定状态', '填装定位器放开状态', '物料抓取数', '填装旋转数', '填装下降数', '填装数',
       '加盖检测数', '加盖定位数', '推盖数', '加盖下降数', '加盖数', '拧盖检测数', '拧盖定位数', '拧盖下降数',
       '拧盖旋转数', '拧盖数', '合格数', '不合格数', '物料推送装置故障1001', '物料检测装置故障2001',
       '填装装置检测故障4001', '填装装置定位故障4002', '填装装置填装故障4003', '加盖装置定位故障5001',
       '加盖装置加盖故障5002', '拧盖装置定位故障6001', '拧盖装置拧盖故障6002'],
      dtype='object')

In [6]:
df

,日期,物料推送气缸推送状态,物料推送气缸收回状态,物料推送数,物料待抓取数,放置容器数,容器上传检测数,填装检测数,填装定位器固定状态,填装定位器放开状态,...,不合格数,物料推送装置故障1001,物料检测装置故障2001,填装装置检测故障4001,填装装置定位故障4002,填装装置填装故障4003,加盖装置定位故障5001,加盖装置加盖故障5002,拧盖装置定位故障6001,拧盖装置拧盖故障6002
时间,,,,,,,,,,,,,,,,,,,,,
9759,1,1,1,1464,1464,491,490,488,0,1,...,0,1001,0,0,0,0,0,0,0,0
9760,1,1,1,1464,1464,491,490,488,0,1,...,0,1001,0,0,0,0,0,0,0,0
9761,1,1,1,1464,1464,491,490,488,0,1,...,0,1001,0,0,0,0,0,0,0,0
9762,1,1,1,1464,1464,491,490,488,0,1,...,0,1001,0,0,0,0,0,0,0,0
9763,1,1,1,1464,1464,491,490,488,0,1,...,0,1001,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10033,2,1,1,1461,1461,490,489,488,1,0,...,0,1001,0,0,0,0,0,0,0,0
10034,2,1,1,1461,1461,490,489,488,1,0,...,0,1001,0,0,0,0,0,0,0,0
10035,2,1,1,1461,1461,490,489,488,1,0,...,0,1001,0,0,0,0,0,0,0,0


In [7]:
n = len(df.columns)

# 选择倒数前九列
dfx = df.iloc[:, :n-9]

# df_trimmed 现在包含了 df 的倒数前九列的内容
dfx

,日期,物料推送气缸推送状态,物料推送气缸收回状态,物料推送数,物料待抓取数,放置容器数,容器上传检测数,填装检测数,填装定位器固定状态,填装定位器放开状态,...,推盖数,加盖下降数,加盖数,拧盖检测数,拧盖定位数,拧盖下降数,拧盖旋转数,拧盖数,合格数,不合格数
时间,,,,,,,,,,,,,,,,,,,,,
9759,1,1,1,1464,1464,491,490,488,0,1,...,487,487,487,487,487,487,487,486,486,0
9760,1,1,1,1464,1464,491,490,488,0,1,...,487,487,487,487,487,487,487,486,486,0
9761,1,1,1,1464,1464,491,490,488,0,1,...,487,487,487,487,487,487,487,486,486,0
9762,1,1,1,1464,1464,491,490,488,0,1,...,487,487,487,487,487,487,487,487,486,0
9763,1,1,1,1464,1464,491,490,488,0,1,...,487,487,487,487,487,487,487,487,486,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10033,2,1,1,1461,1461,490,489,488,1,0,...,487,487,487,486,486,486,486,486,485,0
10034,2,1,1,1461,1461,490,489,488,1,0,...,487,487,487,486,486,486,486,486,485,0
10035,2,1,1,1461,1461,490,489,488,1,0,...,487,487,487,486,486,486,486,486,485,0


In [8]:
# 将特定列中所有不等于 0 的值替换为 1
df['填装装置填装故障4003'] = (df['填装装置填装故障4003'] != 0).astype(int)
df['填装装置填装故障4003'].value_counts()

填装装置填装故障4003
0    57486
1     2760
Name: count, dtype: int64

In [9]:
X=dfx
y=df['填装装置填装故障4003']
# 划分训练集和测试集

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [10]:
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# from sklearn.metrics import classification_report, confusion_matrix
# # 转换为DMatrix对象，这是XGBoost的优化数据结构
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test, label=y_test)

# # 设置参数，这里是一些基本的参数，可以根据任务调整
# param = {
#     'max_depth': 3,  # 树的最大深度
#     'eta': 0.1,      # 学习率
#     'objective': 'binary:logistic',  # 二分类的逻辑回归问题
#     'eval_metric': ['logloss', 'aucpr', 'auc', 'map']  # 评估指标
    
# }
# num_round = 1000  # 训练轮数

# # 训练模型
# bst = xgb.train(param, dtrain, num_round)

# # 预测测试集
# preds = bst.predict(dtest)
# predictions = [round(value) for value in preds]

# from sklearn.metrics import precision_score, recall_score

# # # 计算精确度和召回率
# # precision = precision_score(y_test, predictions)
# # recall = recall_score(y_test, predictions)

# # # 打印评估结果
# # print(f"Accuracy: {accuracy * 100.0}%")
# # print(f"Precision: {precision * 100.0}%")
# # print(f"Recall: {recall * 100.0}%")
# # 生成混淆矩阵
# cm = confusion_matrix(y_test, predictions)
# print("Confusion Matrix:")
# print(cm)

# # 生成分类报告
# report = classification_report(y_test, predictions, target_names=['0', '1'])
# print("Classification Report:")
# print(report)


In [11]:
xgb1=XGBClassifier(random_state=120,
                  objective = 'multi:softmax',
                  eval_metric = ['mlogloss', 'aucpr', 'auc', 'map'],
                  num_class=2,
                  n_estimators=340,
                  reg_lambda=2,
                  min_child_weight=0,
                  max_depth=6,
                  learning_rate=0.1)

In [12]:
from sklearn.model_selection import KFold
cv =KFold(n_splits=3,shuffle=True,random_state=120)

In [13]:
from sklearn.model_selection import cross_validate

result = cross_validate(xgb1,X,y,cv=cv,
                       return_train_score=True,
                       verbose=True)
result

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   15.3s finished


{'fit_time': array([4.74827361, 5.10806417, 5.20922995]),
 'score_time': array([0.04821467, 0.04001093, 0.04399681]),
 'test_score': array([0.99935265, 0.99950204, 0.99950204]),
 'train_score': array([0.99967633, 0.99960163, 0.99952694])}

In [14]:
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report
import xgboost as xgb

# 假设您的模型是 xgb1，数据是 X, y
# 需要确保模型 xgb1 已经定义和训练好

# 使用交叉验证生成预测值
y_pred = cross_val_predict(xgb1, X, y, cv=cv)
# 计算混淆矩阵
cm = confusion_matrix(y, y_pred)
print("Confusion Matrix:")
print(cm)

# 生成分类报告
report = classification_report(y, y_pred, target_names=['Class 0', 'Class 1'])
print("Classification Report:")
print(report)


Confusion Matrix:
[[57453    33]
 [    0  2760]]
Classification Report:
              precision    recall  f1-score   support

     Class 0       1.00      1.00      1.00     57486
     Class 1       0.99      1.00      0.99      2760

    accuracy                           1.00     60246
   macro avg       0.99      1.00      1.00     60246
weighted avg       1.00      1.00      1.00     60246

